In [1]:
import pandas as pd
from IPython.display import display, HTML
from assembly_tools import load_experimental_grid, format_table, extract_raw_performance
import itertools
from os import listdir
from os.path import isfile, join

In [2]:
from hydra import initialize, compose
from omegaconf import OmegaConf

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# load default params. They can be changed however you want.
with initialize(version_base=None, config_path="config"):
    cfg = compose(config_name='extract.yaml')
    print(cfg)

{'metrics': ['Max Acc thresh', 'Max Acc', 'Max individual Acc thresh', 'Max individual Acc', 'Null Acc', 'Max F1 thresh', 'Max F1', 'Max individual F1 thresh', 'Max individual F1', 'Null F1', 'AUROC', 'Individual AUROC', 'Null AUROC', 'runtime'], 'hp_list': ['data_preprocess.resolution', 'data_preprocess.normalize', 'label_path', 'method.max_lag', 'method.var_absolute_values', 'method.filter_mode'], 'rename': {'resolution_value': 'data_preprocess.resolution', 'method_hps.var_absolute_values': 'method.var_absolute_values', 'method_hps.filter_mode': 'method.filter_mode', 'method_hps.max_lag': 'method.max_lag', 'ds_name': 'label_path', 'normalize': 'data_preprocess.normalize'}, 'ds_order': ['close_3', 'close_5', 'root_cause_3', 'root_cause_5', '1_random_3', '1_random_5', 'confounder_3', 'confounder_5', 'random_3', 'random_5', 'disjoint_10'], 'data_path': '/home/datasets4/stein/rivers/paper_result_storage/new_runs/exp1/', 'method_name': 'combo', 'save_path': 'grid_export'}


In [46]:
data["window_data_month_value"] = data["window_data_month_value"].astype(str)


In [49]:
# test for consistency
# Fix old hydra specs. Does nothing if strictly  version is used.
for x in cfg.rename:
    if cfg.rename[x] in data.columns:
        data.loc[data[cfg.rename[x]].isnull(), cfg.rename[x]] = data.loc[
            data[cfg.rename[x]].isnull(), x
        ]
    data.drop(columns=x, inplace=True)
print("Number of samples in grid:", len(data))

Number of samples in grid: 600


In [54]:
# TODO Fix when relevant for exp2
data["window_data_month_value"] = data["window_data_month_value"].astype(str)
# nothing should vary outside of the hps and the metrics.
control = data[
    [x for x in data.columns if ((x not in cfg.metrics) and (x not in cfg.hp_list))]
]
control = control.T[control.nunique() > 1]

# Whatever remains here should come from inconsistencies.
print("Validate! The following additional columns have non unique values:")
for x in control.T.columns:
    print(control.T[x].value_counts())

Validate! The following additional columns have non unique values:
data_path
resources/rivers_ts_east_germany.csv        540
../../product/rivers_ts_east_germany.csv     60
Name: count, dtype: int64
window_data_month_value
[]     540
nan     60
Name: count, dtype: int64
restrict_to
False    540
-1        60
Name: count, dtype: int64


In [56]:
relevant_hps = [x for x in cfg.hp_list if x in data.columns]
# label path parsing
str_data_path = data["label_path"].str.contains("/")
data.loc[str_data_path, "label_path"] = [
    x[3] for x in data[str_data_path]["label_path"].str.split("/").values
]
data = data[cfg.metrics + [x for x in cfg.hp_list if x in data.columns]]

print("Dropping duplicates:")
print(len(data))
data.drop_duplicates()
print(len(data))

Dropping duplicates:
600
600


In [59]:
assert int(data[relevant_hps].duplicated().sum()) == 0, "Duplicated columns!!"


AssertionError: Duplicated columns!!

In [62]:
int(data[relevant_hps].duplicated().sum())

270

In [76]:
data[[x for x in data.columns if x != "runtime"]].sort_values(["data_preprocess.resolution",	"data_preprocess.normalize","label_path",	"method.filter_mode"]).drop_duplicates()

,Max Acc thresh,Max Acc,Max individual Acc thresh,Max individual Acc,Null Acc,Max F1 thresh,Max F1,Max individual F1 thresh,Max individual F1,Null F1,AUROC,Individual AUROC,Null AUROC,data_preprocess.resolution,data_preprocess.normalize,label_path,method.filter_mode
2024-11-02 18:16:50.9408,True,0.8333333333333334,0.15668202764976957,0.8333333333333331,0.8333333333333334,True,0.5622119815668203,1.0,0.6069782751810402,0.2857142857142857,0.8059907834101382,0.8059907834101383,0.5,12H,False,1_random_3,biggest
2024-11-02 18:16:28.9673,True,0.8333333333333334,0.3118279569892473,0.8333333333333331,0.8333333333333334,True,0.45878136200716857,1.0,0.5478750640040962,0.2857142857142857,0.7129032258064516,0.7129032258064516,0.5,12H,False,1_random_3,next
2024-11-02 18:16:07.4755,True,0.8333333333333334,1.0,0.8333333333333331,0.8333333333333334,True,0.4477726574500768,1.0,0.4776168531928901,0.2857142857142857,0.7373271889400923,0.7373271889400922,0.5,12H,False,1_random_3,none
2024-11-02 18:51:45.2199,True,0.8493010752688172,0.432258064516129,0.8663978494623654,0.8493010752688172,True,0.4841330676069293,0.9985663082437276,0.4950091646445082,0.2619258982385647,0.7138964729345346,0.7141054360812424,0.5,12H,False,1_random_5,biggest
2024-11-02 18:45:52.7340,True,0.8493010752688172,0.5641577060931899,0.8589784946236556,0.8493010752688172,True,0.39102662374163216,0.9935483870967742,0.4340653917792768,0.2619258982385647,0.6501167901499834,0.6505095228055379,0.5,12H,False,1_random_5,next
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-02 02:38:09.3716,True,0.6653826399589111,0.4406779661016949,0.7768361581920905,0.6653826399589111,False,0.5014431402732346,0.7750385208012327,0.667899332306112,0.5014431402732346,0.5840067735502166,0.5841037493579866,0.5,6H,True,root_cause_3,next
2024-11-02 02:37:10.9207,True,0.6653826399589111,0.6610169491525424,0.7223934257832563,0.6653826399589111,False,0.5014431402732346,0.423728813559322,0.6016435541859271,0.5014431402732346,0.5925612390490371,0.5925783256291731,0.5,6H,True,root_cause_3,none
2024-11-02 02:53:11.7450,True,0.7993129770992367,0.9969465648854962,0.7993129770992365,0.7993129770992367,False,0.3342869858223663,0.1251908396946565,0.33438507209499585,0.3342869858223663,0.5121823904664657,0.5122137404580153,0.5,6H,True,root_cause_5,biggest
2024-11-02 02:52:45.3049,True,0.7993129770992367,0.716030534351145,0.8148854961832059,0.7993129770992367,False,0.3342869858223663,0.6137404580152672,0.4199491094147583,0.3342869858223663,0.5478729875361896,0.5479262086513995,0.5,6H,True,root_cause_5,next


In [71]:
data[data[relevant_hps].duplicated()].sort_values(["data_preprocess.resolution",	"data_preprocess.normalize","label_path",	"method.filter_mode"])

,Max Acc thresh,Max Acc,Max individual Acc thresh,Max individual Acc,Null Acc,Max F1 thresh,Max F1,Max individual F1 thresh,Max individual F1,Null F1,AUROC,Individual AUROC,Null AUROC,runtime,data_preprocess.resolution,data_preprocess.normalize,label_path,method.filter_mode
2024-11-02 18:17:54.3439,True,0.8333333333333334,0.15668202764976957,0.8333333333333331,0.8333333333333334,True,0.5622119815668203,1.0,0.6069782751810402,0.2857142857142857,0.8059907834101382,0.8059907834101383,0.5,0 days 00:00:19.142244,12H,False,1_random_3,biggest
2024-11-02 18:17:33.5917,True,0.8333333333333334,0.3118279569892473,0.8333333333333331,0.8333333333333334,True,0.45878136200716857,1.0,0.5478750640040962,0.2857142857142857,0.7129032258064516,0.7129032258064516,0.5,0 days 00:00:19.398592,12H,False,1_random_3,next
2024-11-02 18:17:12.5817,True,0.8333333333333334,1.0,0.8333333333333331,0.8333333333333334,True,0.4477726574500768,1.0,0.4776168531928901,0.2857142857142857,0.7373271889400923,0.7373271889400922,0.5,0 days 00:00:19.411457,12H,False,1_random_3,none
2024-11-02 18:45:44.7704,True,0.8493010752688172,0.432258064516129,0.8663978494623654,0.8493010752688172,True,0.4841330676069293,0.9985663082437276,0.4950091646445082,0.2619258982385647,0.7138964729345346,0.7141054360812424,0.5,0 days 00:05:33.764040,12H,False,1_random_5,biggest
2024-11-02 18:53:26.7680,True,0.8493010752688172,0.5641577060931899,0.8589784946236556,0.8493010752688172,True,0.39102662374163216,0.9935483870967742,0.4340653917792768,0.2619258982385647,0.6501167901499834,0.6505095228055379,0.5,0 days 00:07:28.607848,12H,False,1_random_5,next
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-02 02:41:10.4956,True,0.6653826399589111,0.4406779661016949,0.7768361581920905,0.6653826399589111,False,0.5014431402732346,0.7750385208012327,0.667899332306112,0.5014431402732346,0.5840067735502166,0.5841037493579866,0.5,0 days 00:00:27.441017,6H,True,root_cause_3,next
2024-11-02 02:40:09.6642,True,0.6653826399589111,0.6610169491525424,0.7223934257832563,0.6653826399589111,False,0.5014431402732346,0.423728813559322,0.6016435541859271,0.5014431402732346,0.5925612390490371,0.5925783256291731,0.5,0 days 00:00:26.999491,6H,True,root_cause_3,none
2024-11-02 02:51:11.9784,True,0.7993129770992367,0.9969465648854962,0.7993129770992365,0.7993129770992367,False,0.3342869858223663,0.1251908396946565,0.33438507209499585,0.3342869858223663,0.5121823904664657,0.5122137404580153,0.5,0 days 00:00:29.684213,6H,True,root_cause_5,biggest
2024-11-02 02:51:16.0705,True,0.7993129770992367,0.716030534351145,0.8148854961832059,0.7993129770992367,False,0.3342869858223663,0.6137404580152672,0.4199491094147583,0.3342869858223663,0.5478729875361896,0.5479262086513995,0.5,0 days 00:01:02.772818,6H,True,root_cause_5,next


In [73]:
data.sort_values(["data_preprocess.resolution",	"data_preprocess.normalize","label_path",	"method.filter_mode"])

,Max Acc thresh,Max Acc,Max individual Acc thresh,Max individual Acc,Null Acc,Max F1 thresh,Max F1,Max individual F1 thresh,Max individual F1,Null F1,AUROC,Individual AUROC,Null AUROC,runtime,data_preprocess.resolution,data_preprocess.normalize,label_path,method.filter_mode
2024-11-02 18:16:50.9408,True,0.8333333333333334,0.15668202764976957,0.8333333333333331,0.8333333333333334,True,0.5622119815668203,1.0,0.6069782751810402,0.2857142857142857,0.8059907834101382,0.8059907834101383,0.5,0 days 00:00:19.775793,12H,False,1_random_3,biggest
2024-11-02 18:17:54.3439,True,0.8333333333333334,0.15668202764976957,0.8333333333333331,0.8333333333333334,True,0.5622119815668203,1.0,0.6069782751810402,0.2857142857142857,0.8059907834101382,0.8059907834101383,0.5,0 days 00:00:19.142244,12H,False,1_random_3,biggest
2024-11-02 18:16:28.9673,True,0.8333333333333334,0.3118279569892473,0.8333333333333331,0.8333333333333334,True,0.45878136200716857,1.0,0.5478750640040962,0.2857142857142857,0.7129032258064516,0.7129032258064516,0.5,0 days 00:00:19.798040,12H,False,1_random_3,next
2024-11-02 18:17:33.5917,True,0.8333333333333334,0.3118279569892473,0.8333333333333331,0.8333333333333334,True,0.45878136200716857,1.0,0.5478750640040962,0.2857142857142857,0.7129032258064516,0.7129032258064516,0.5,0 days 00:00:19.398592,12H,False,1_random_3,next
2024-11-02 18:16:07.4755,True,0.8333333333333334,1.0,0.8333333333333331,0.8333333333333334,True,0.4477726574500768,1.0,0.4776168531928901,0.2857142857142857,0.7373271889400923,0.7373271889400922,0.5,0 days 00:00:19.234775,12H,False,1_random_3,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-02 02:51:11.9784,True,0.7993129770992367,0.9969465648854962,0.7993129770992365,0.7993129770992367,False,0.3342869858223663,0.1251908396946565,0.33438507209499585,0.3342869858223663,0.5121823904664657,0.5122137404580153,0.5,0 days 00:00:29.684213,6H,True,root_cause_5,biggest
2024-11-02 02:52:45.3049,True,0.7993129770992367,0.716030534351145,0.8148854961832059,0.7993129770992367,False,0.3342869858223663,0.6137404580152672,0.4199491094147583,0.3342869858223663,0.5478729875361896,0.5479262086513995,0.5,0 days 00:01:01.969855,6H,True,root_cause_5,next
2024-11-02 02:51:16.0705,True,0.7993129770992367,0.716030534351145,0.8148854961832059,0.7993129770992367,False,0.3342869858223663,0.6137404580152672,0.4199491094147583,0.3342869858223663,0.5478729875361896,0.5479262086513995,0.5,0 days 00:01:02.772818,6H,True,root_cause_5,next
2024-11-02 02:49:44.6801,True,0.7993129770992367,1.0,0.7993129770992365,0.7993129770992367,True,0.3405599738533609,0.40610687022900765,0.3817666303162487,0.3342869858223663,0.5591274224788425,0.5591221374045802,0.5,0 days 00:01:01.565273,6H,True,root_cause_5,none


### Assembly of experimental results 1

In [87]:
# We conduced a large HP search to determine which HP configurations are optimal for a certain method and dataset. 
# You can un the same HP searches by using the benchmarking script in the causal discovery zoo folder. 
# Check the readme for details. You can also reproduce the actual numbers by running the selected HP combo in the same way.
# YOu can also use the functions here directly to summarize your own custom hp searches.
# NOTABLY THE EXACT HP NAMING was changed slightly due to changes in the slurm configurations. the exported params however can be directly used.

#TODO: 

# 5 random runs on dynotears broken.

In [114]:
with initialize(version_base=None, config_path="config/"):
    cfg = compose(config_name='extract.yaml')
scoring = "Max individual F1"
general_params =  ["resolution_value", "ds_name", "method_hps.max_lag"]
hp_stack = []
performance_stack = []

In [116]:
for name in ["reverse", "cross", "combo", ]:
    data = pd.read_csv("grid_export_exp1/" + name + ".csv", index_col=0)
    # quick fix
    res = "method.filter_mode"

    data.loc[data[res].isnull(),res] = "none"
    formatted = format_table(data,cfg, scoring= scoring, restriction=[res] if res else [])
    lines = extract_raw_performance(formatted,cfg,scoring,name=name, restriction=[res])
    hp_stack.append(formatted)
    performance_stack.append(lines)


In [106]:
for name in ["var_", "varlingam", "dynotears"]:
    data = pd.read_csv("grid_export_exp1/" + name + ".csv", index_col=0)
    # quick fix
    formatted = format_table(data,cfg, scoring= scoring, restriction=[res] if res else [])
    lines = extract_raw_performance(formatted,cfg,scoring,name=name, restriction=None)
    hp_stack.append(formatted)
    performance_stack.append(lines)

In [121]:
for name in ["cp"]:
    data = pd.read_csv("grid_export_exp1/" + name + ".csv", index_col=0)
    # quick fix
    res = "method.architecture"
    cfg.ds_order = cfg.ds_order[:-1]
    formatted = format_table(data,cfg, scoring= scoring, restriction=[res] if res else [])
    lines = extract_raw_performance(formatted,cfg,scoring,name=name, restriction=[res])
    hp_stack.append(formatted)
    performance_stack.append(lines)

In [123]:
data = load_experimental_grid(mypath=cfg.data_path + cfg.exp, method_name="cdmi_confounder_3").T


In [ ]:
data.groupby("ds")

In [125]:
data.columns

Index(['Max Acc thresh', 'Max Acc', 'Max individual Acc thresh',
       'Max individual Acc', 'Null Acc', 'Max F1 thresh', 'Max F1',
       'Max individual F1 thresh', 'Max individual F1', 'Null F1', 'AUROC',
       'Individual AUROC', 'Null AUROC', 'sample_path', 'data_path',
       'full_graph_path', 'save_path', 'ds_name', 'eval_set',
       'resolution_value', 'subsample_value', 'window_data_year_value',
       'window_data_month_value', 'interpolation', 'normalize',
       'save_full_out', 'subsample_strategy', 'var_test_window', 'restrict_to',
       'parallel', 'remove_diagonal', 'method_hps.method',
       'method_hps.map_to_summary_graph', 'method_hps.max_lag',
       'method_hps.ci_test', 'method_hps.cut_at', 'method_hps.cp_architecture',
       'method_hps.batch_size', 'method_hps.freq', 'method_hps.device',
       'method_hps.learning_rate', 'method_hps.epochs',
       'method_hps.num_layers', 'method_hps.num_cells',
       'method_hps.dropout_rate', 'method_hps.context_len

In [140]:
null_model = varl.T[["Null F1", "ds_name"]].groupby("ds_name").max().T
null_model.index = ["Null model"]
null_model.index.name = None
null_model = null_model[order]
null_model.columns = varl_style.columns

In [179]:
out = pd.concat([null_model,cross_style,reverse_style,combo_style, var_style, dyno_style,varl_style,pcmci_style, cp_style]).astype(float)
# fix pandas issue with columns
out.loc["CDMI"] = cdmi_style.values[0]
out = out.iloc[[0,1,2,4,3,5,9,8,7,6,10,12,13,11,16,14,15]]

# Styling of the table

In [76]:
out = pd.concat(performance_stack)

In [77]:
out.rename(index={
"CC_none": "CC",
"CC_corr": "CC+C",
"RP_none": "RP",
"RP_biggest": "RP+B",
"RP_next": "RP+N",
"RPCC_none": "RPCC",
"RPCC_biggest": "RPCC+B",
"RPCC_next": "RPCC+N",
"RPCC_corr": "RPCC+C",
"cp__unidirectional": "CP (Gru)",
"cp__transformer":"CP (Transf)"
}, inplace=True)

In [78]:
out

,close_3,close_5,root_cause_3,root_cause_5,1_random_3,1_random_5,confounder_3,confounder_5,random_3,random_5,disjoint_10
RP+B,0.732495,0.574395,0.502311,0.334385,0.606978,0.495018,0.5625,0.468028,0.706104,0.528594,0.243961
RP,0.746873,0.531817,0.714997,0.47843,0.477617,0.420279,0.663889,0.482844,0.741332,0.522066,0.288077
RP+N,0.675887,0.456021,0.855521,0.672316,0.54846,0.434168,0.551389,0.452997,0.692921,0.490121,0.362898
Combo_corr,0.681238,0.516092,0.677966,0.539589,0.554531,0.472782,0.618056,0.468018,0.684225,0.522022,0.547058
Combo_next,0.652289,0.438106,0.725526,0.583473,0.520225,0.431259,0.598611,0.435546,0.669482,0.489652,0.381007
Combo_biggest,0.647066,0.480611,0.551258,0.364617,0.559213,0.444835,0.604167,0.427995,0.654264,0.492502,0.246316
Combo_none,0.660799,0.471672,0.640164,0.448482,0.503182,0.418532,0.620833,0.443069,0.672074,0.508363,0.302482
CC,0.65461,0.437578,0.664612,0.459444,0.442396,0.385854,0.645833,0.42979,0.67408,0.492855,0.246443
CC+C,0.665957,0.489991,0.702003,0.572397,0.523151,0.462193,0.6,0.449984,0.680184,0.519335,0.529981
VAR,0.819347,0.67533,0.795931,0.617236,0.770851,0.599938,0.773611,0.603825,0.82982,0.649182,0.483831


### Join to latex table

In [23]:
style = out.style.background_gradient(cmap="Greens_r",low=0,high=1)
style = style.format("{:.2f}")
latex_str = style.to_latex(convert_css=True)
display(HTML(style.to_html()))

,close_3,close_5,root_cause_3,root_cause_5,1_random_3,1_random_5,confounder_3,confounder_5,random_3,random_5,disjoint_10
VAR,0.82,0.68,0.80,0.62,0.77,0.60,0.77,0.60,0.83,0.65,0.48
RP+B,0.73,0.57,0.50,0.33,0.61,0.50,0.56,0.47,0.71,0.53,0.24
RP,0.75,0.53,0.71,0.48,0.48,0.42,0.66,0.48,0.74,0.52,0.29
RP+N,0.68,0.46,0.86,0.67,0.55,0.43,0.55,0.45,0.69,0.49,0.36


In [18]:
print(latex_str)

\begin{tabular}{llllllllllll}
 & close_3 & close_5 & root_cause_3 & root_cause_5 & 1_random_3 & 1_random_5 & confounder_3 & confounder_5 & random_3 & random_5 & disjoint_10 \\
VAR & 0.82 & 0.68 & 0.80 & 0.62 & 0.77 & 0.60 & 0.77 & 0.60 & 0.83 & 0.65 & 0.48 \\
RP+B & 0.73 & 0.57 & 0.50 & 0.33 & 0.61 & 0.50 & 0.56 & 0.47 & 0.71 & 0.53 & 0.24 \\
RP & 0.75 & 0.53 & 0.71 & 0.48 & 0.48 & 0.42 & 0.66 & 0.48 & 0.74 & 0.52 & 0.29 \\
RP+N & 0.68 & 0.46 & 0.86 & 0.67 & 0.55 & 0.43 & 0.55 & 0.45 & 0.69 & 0.49 & 0.36 \\
\end{tabular}

